In [1]:
#auto reload 
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
import numpy as np
import os 
from datasets import load_dataset, load_from_disk
import torch 

/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import pandas as pd 
 
bow_english = pd.read_csv("data/unigram_freq_en.csv")
bow_french = pd.read_csv("data/unigram_freq_fr.csv")

In [8]:
train_len, val_len = 10000, 1000

In [9]:
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]

if train_len is not None:
    train_data = train_data.select(range(train_len))
if val_len is not None:
    val_data = val_data.select(range(val_len))


/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [10]:
examples = 4 
#check some data and their translation
for i, t in zip(range(examples), train_data):
    ex = t["translation"]
    print(ex["fr"], " -> ", ex["en"], "\n\n")

Reprise de la session  ->  Resumption of the session 


Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.  ->  I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. 


Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.  ->  Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. 


Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.  ->  You 

In [11]:
from sacremoses import MosesTokenizer, MosesDetokenizer

# Sample sentence
sentence = "This is an  example sentence with words that may not be in the shortlist."

mt_en = MosesTokenizer(lang="en")
mt_fr = MosesTokenizer(lang="fr")

tokenized = mt_en.tokenize(sentence)
tokenized


['This',
 'is',
 'an',
 'example',
 'sentence',
 'with',
 'words',
 'that',
 'may',
 'not',
 'be',
 'in',
 'the',
 'shortlist',
 '.']

In [12]:
class TokenizerWrapper:
    def __init__(self, tokenizer_en, tokenizer_fr):
        self.tokenizer_en = tokenizer_en
        self.tokenizer_fr = tokenizer_fr
    
    def tokenize_function(self, examples):
        return {"tokenized_en" : self.tokenizer_en.tokenize(examples["translation"]["en"]),
                "tokenized_fr" : self.tokenizer_fr.tokenize(examples["translation"]["fr"])}
    
tokenizer_wrapper = TokenizerWrapper(mt_en, mt_fr)


In [13]:
if not os.path.exists("processed_data/tokenized_train_data{}".format(train_len)):
    tokenized_train_data  = train_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_train_data.save_to_disk("processed_data/tokenized_train_data{}".format(train_len))


Map (num_proc=19):   0%|          | 0/10000 [00:00<?, ? examples/s]/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Users/nadia/anaconda3/envs/projetmla/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/Users/nadia/anaconda3/envs/projetmla/lib/python3

In [14]:
    
if not os.path.exists("processed_data/tokenized_val_data{}".format(val_len)):
    tokenized_val_data  = val_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_val_data.save_to_disk("processed_data/tokenized_val_data{}".format(val_len))


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 199671.71 examples/s]


In [15]:
tokenized_train_data = load_from_disk("processed_data/tokenized_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/tokenized_val_data{}".format(val_len))

In [12]:
# import pandas as pd
# df = pd.read_csv("data/Copy of Lexique383.csv")
# df.sort_values(by="9_freqfilms2", ascending=False, inplace=True)
# only_words_freq = df[["1_ortho", "9_freqfilms2"]][:30000]
# #drop index
# only_words_freq.reset_index(drop=True, inplace=True)
# #change column names
# only_words_freq.columns = ["word", "count"]
# only_words_freq.to_csv("data/unigram_freq_fr.csv", index=False)

In [16]:
most_frequent_english_words = bow_english["word"][:30000].apply(lambda x: str(x)).tolist()
most_frequent_english_words.append("UNK")
most_frequent_french_words = bow_french["word"][:30000].apply(lambda x: str(x)).tolist()
most_frequent_french_words.append("UNK")
tokenized_most_frequent_english_words = mt_en.tokenize(" ".join(most_frequent_english_words))
tokenized_most_frequent_french_words = mt_fr.tokenize(" ".join(most_frequent_french_words))

In [17]:
class toIdTransform:
    def __init__(self, most_frequent_words, tor):
        self.most_frequent_words = most_frequent_words
        self.word_to_id = {word: i for i, word in enumerate(most_frequent_words)}
        self.tor = tor
    
    def __call__(self, tokenized):
        return {"ids_en":[self.word_to_id.get(token, self.word_to_id["UNK"]) for token in tokenized["tokenized_en"]],
                "ids_fr":[self.word_to_id.get(token, self.word_to_id["UNK"]) for token in tokenized["tokenized_fr"]]}
    
to_id_transform = toIdTransform(tokenized_most_frequent_english_words, torch)


In [18]:
if not os.path.exists("processed_data/id_train_data{}".format(train_len)):
    tokenized_train_data = tokenized_train_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_train_data.save_to_disk("processed_data/id_train_data{}".format(train_len))


Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 867739.16 examples/s]


In [19]:
if not os.path.exists("processed_data/id_val_data{}".format(val_len)):
    tokenized_val_data = tokenized_val_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_val_data.save_to_disk("processed_data/id_val_data{}".format(val_len))

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 193571.35 examples/s]


In [20]:
tokenized_train_data = load_from_disk("processed_data/id_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/id_val_data{}".format(val_len))

In [21]:
class to_tensor:
    def __init__(self, tor):
        self.tor = tor
    
    def __call__(self, ids):
        return {"ids_en":self.tor.tensor(ids["ids_en"]),
                "ids_fr":self.tor.tensor(ids["ids_fr"])}
    
tokenized_train_data = tokenized_train_data.map(to_tensor(torch), batched=False, num_proc=19)
tokenized_val_data = tokenized_val_data.map(to_tensor(torch), batched=False, num_proc=19)

Map (num_proc=19): 100%|██████████| 1000/1000 [00:00<00:00, 6781.27 examples/s]


In [24]:
def to_vector(ids, vocab_size):
    vecs = torch.zeros(len(ids), vocab_size)
    vecs[range(len(ids)), ids] = 1
    return vecs

class to_vec:
    def __init__(self, tor, vocab_size, lang):
        self.tor = tor
        self.vocab_size = vocab_size
        self.lang = lang

    
    def __call__(self, ids):
        ids = self.tor.tensor(ids["ids_{}".format(self.lang)])
        vecs = self.tor.zeros(len(ids), self.vocab_size)
        vecs[range(len(ids)), ids] = 1
        return vecs

In [27]:
# to_vec_transform_en = to_vec(torch, len(tokenized_most_frequent_english_words), "en")
# to_vec_transform_fr = to_vec(torch, len(tokenized_most_frequent_french_words), "fr")
# vecs_en = [

# ]
# vecs_fr = [

# ]
# for i in range(len(tokenized_train_data)):
#     vecs_en.append(to_vec_transform_en(tokenized_train_data[i]))
#     vecs_fr.append(to_vec_transform_fr(tokenized_train_data[i]))



In [25]:
from models.fcnn import FCNN

model = FCNN(len(tokenized_most_frequent_english_words), [10,10], 30000, "cuda" if torch.cuda.is_available() else "cpu",torch.nn.ReLU())

In [26]:
for i in range(len(tokenized_train_data)):

    tensorized = torch.tensor(tokenized_train_data[i]["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    
    result = model(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu"))
    
    print(result.shape)
    
    break

torch.Size([4, 30000])


In [27]:
from models.rnn import RNN

encoder_rnn = RNN(len(tokenized_most_frequent_english_words),100,2,"cuda" if torch.cuda.is_available() else "cpu",torch.nn.Tanh())

In [40]:
for x in tokenized_train_data:
    tensorized = torch.tensor(x["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    result = encoder_rnn(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu").view(1,-1,len(tokenized_most_frequent_english_words)))
    print(result[0].shape)
    break
print (encoder_rnn)

torch.Size([1, 4, 100])
RNN(
  (rnn): RNN(30001, 100, num_layers=2, batch_first=True)
)


In [41]:
from models.rnn import RNN
from models.enco_deco import Encoder, Decoder  

# Importing necessary modules

# Création de l'encodeur (Encoder creation)
encoder = Encoder(len(tokenized_most_frequent_english_words), 256, 512, 2, "cuda" if torch.cuda.is_available() else "cpu", 0.5)

# Printing the length of most frequent English words and the encoder details
print(len(tokenized_most_frequent_english_words))
print(encoder)

# Looping through the tokenized training data
for x in tokenized_train_data:
    # Converting English word indices to a PyTorch tensor
    tensorized = torch.tensor(x["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu").long()
    print(tensorized)

    # Applying the encoder to the input tensor
    result = encoder(to_vector(tensorized, len(tokenized_most_frequent_english_words))
                     .to("cuda" if torch.cuda.is_available() else "cpu")
                     .view(1, -1, len(tokenized_most_frequent_english_words)))
    print(result[0].shape)
    break

# Création du décodeur (Decoder creation)
decoder = Decoder(len(tokenized_most_frequent_english_words), 100, 2, "cuda" if torch.cuda.is_available() else "cpu", torch.nn.Tanh())

# Using the decoder on some input
decoded_output, hidden_state = decoder(torch.tensor([1]), result[0], None)

# Printing the shapes of the decoded output and hidden state
print("Decoded Output shape:", decoded_output.shape)
print("Hidden State shape:", hidden_state.shape)


30001
Encoder(
  (embedding): Embedding(30001, 256)
  (rnn): RNN(
    (rnn): RNN(256, 512, num_layers=2, batch_first=True, dropout=0.5)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)
tensor([30000,     1,     0,  1497])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)